# Generating texts
In this notebook we will generate texts and store them in a CSV file. More precisely, we generate random PhD topics which could potentially be worked on at any of the institutes of the [Leibnitz MMS network](https://www.wias-berlin.de/research/Leibniz-MMS/Netzwerkpartner.jsp?lang=0).

In [1]:
import bia_bob
import random

First, we test the connection to the LLM server.

In [2]:
def prompt_scadsai_llm(message:str, model="gemma3:12b", temperature=0.2):
    """A prompt helper function that sends a message to ScaDS.AI LLM server at 
    ZIH TU Dresden and returns only the text response.
    """
    import os
    import openai
    
    # convert message in the right format if necessary
    if isinstance(message, str):
        message = [{"role": "user", "content": message}]
    
    # setup connection to the LLM
    client = openai.OpenAI(base_url="http://localhost:11434/v1",
                           api_key = "none"
            #base_url="https://chat-ai.academiccloud.de/v1",
            #api_key = os.environ.get('KISSKI_API_KEY')
            #base_url="https://llm.scads.ai/v1",
            #api_key=os.environ.get('SCADSAI_API_KEY')
                           # openai/gpt-oss-120b
    )
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature
    )
    
    # extract answer
    return response.choices[0].message.content

prompt_scadsai_llm("hello world")

"Hello there! 👋 \n\nIt's great to see you! Is there anything I can do for you today?\n"

Next, we define a couple of random first and lastnames. We also define a list of institutes.
These will hint the LLM to generate PhD topics that are relevant to a given institute.

In [3]:
import pandas as pd
import numpy as np
import random

# Define separate lists for first names and last names
first_names = ["Alex", "Jordan", "Morgan", "Taylor", "Avery",
               "Sam", "Casey", "Riley", "Jamie", "Dakota",
               "Bailey", "Cameron", "Sydney", "Robin", "Charlie",
               "Dana", "Kris", "Skyler", "Devon", "Reese"]

last_names = ["Reed", "Smith", "Lee", "Garcia", "Patel",
              "Chen", "O'Hara", "Jain", "Kumar", "Singh",
              "Liu", "Davis", "Clark", "Thomas", "Brooks",
              "Flores", "Rivera", "Adams", "Gonzalez", "Campbell"]

faculties = """DIPF - Deutsches Institut für internationale pädagogische Forschung, Frankfurt am Main und Berlin
IDS - Institut für Deutsche Sprache, Mannheim
IWM - Leibniz-Institut für Wissensmedien, Tübingen
ZAS - Leibniz-Zentrum Allgemeine Sprachwissenschaft, Berlin
GESIS - Leibniz-Institut für Sozialwissenschaften, Köln, Mannheim, Berlin
IÖR - Leibniz-Institut für ökologische Raumentwicklung, Dresden
DRFZ - Deutsches Rheuma-Forschungszentrum Berlin
FBN - Forschungsinstitut für Nutztierbiologie, Dummerstorf
HKI - Leibniz-Institut für Naturstoff-Forschung und Infektionsbiologie - Hans-Knöll-Institut, Jena
IPK - Leibniz-Institut für Pflanzengenetik und Kulturpflanzenforschung, Gatersleben
IZW - Leibniz-Institut für Zoo- und Wildtierforschung, Berlin
LIN - Leibniz-Institut für Neurobiologie, Magdeburg
LIR - Leibniz-Institut für Resilienzforschung, Mainz
MfN - Museum für Naturkunde - Leibniz-Institut für Evolutions- und Biodiversitätsforschung, Berlin
SGN - Senckenberg Gesellschaft für Naturforschung, Frankfurt am Main
AIP - Leibniz Institut für Astrophysik, Potsdam
FBH - Ferdinand-Braun-Institut, Leibniz-Institut für Höchstfrequenztechnik, Berlin
FIZ-KA - Leibniz-Institut für Informationsinfrastruktur, Karlsruhe
IAP - Leibniz-Institut für Atmosphärenphysik an der Universität Rostock, Kühlungsborn
IFW - Leibniz-Institut für Festkörper- und Werkstoffforschung Dresden
IHP - Leibniz-Institut für innovative Mikroelektronik, Frankfurt (Oder)
IKZ - Leibniz Institute für Kristallzüchtung, Berlin
INP - Leibniz-Institut für Plasmaforschung und Technologie e.V. Greifswald
IOM - Leibniz-Institut für Oberflächenmodifizierung, Leipzig
IPF - Leibniz Institut für Polymerforschung, Dresden
IPHT - Leibniz-Institut für Photonische Technologien, Jena
ISAS - Leibniz-Institut für Analytische Wissenschaften, Dortmund/Berlin
IVW - Leibniz-Institut für Verbundwerkstoffe, Kaiserslautern
IWT - Leibniz-Institut für Werkstofforientierte Technologien, Bremen
MBI - Max-Born-Institut für Nichtlineare Optik und Kurzzeitspektroskopie, Berlin
MFO - Mathematisches Forschungsinstitut Oberwolfach
PDI - Paul-Drude-Institut für Festkörperelektronik, Berlin
TIB - Leibniz-Informationszentrum Technik und Naturwissenschaften Universitätsbibliothek, Hannover
WIAS - Weierstraß-Institut für Angewandte Analysis und Stochastik, Berlin
ATB - Leibniz-Institut für Agrartechnik und Bioökonomie e.V., Potsdam
IGZ - Leibniz-Institut für Gemüse- und Zierpflanzenbau, Großbeeren und Erfurt
IOW - Leibniz-Institut für Ostseeforschung Warnemünde an der Universität Rostock
LIAG - Institut für Angewandte Geophysik
PIK - Potsdam-Institut für Klimafolgenforschung
TROPOS - Leibniz-Institut für Troposphärenforschung, Leipzig
ZALF - Leibniz-Zentrum für Agrarlandschaftsforschung, Müncheberg""".split("\n")
len(faculties), faculties[:10]

(41,
 ['DIPF - Deutsches Institut für internationale pädagogische Forschung, Frankfurt am Main und Berlin',
  'IDS - Institut für Deutsche Sprache, Mannheim',
  'IWM - Leibniz-Institut für Wissensmedien, Tübingen',
  'ZAS - Leibniz-Zentrum Allgemeine Sprachwissenschaft, Berlin',
  'GESIS - Leibniz-Institut für Sozialwissenschaften, Köln, Mannheim, Berlin',
  'IÖR - Leibniz-Institut für ökologische Raumentwicklung, Dresden',
  'DRFZ - Deutsches Rheuma-Forschungszentrum Berlin',
  'FBN - Forschungsinstitut für Nutztierbiologie, Dummerstorf',
  'HKI - Leibniz-Institut für Naturstoff-Forschung und Infektionsbiologie - Hans-Knöll-Institut, Jena',
  'IPK - Leibniz-Institut für Pflanzengenetik und Kulturpflanzenforschung, Gatersleben'])

In the following loop we select an institute name randomly and ask the LLM server to generate a PhD topic related to the chosen institute.

In [4]:
research_fields = faculties

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

profiles = []

#for field in faculties:
while len(profiles) < 250:    
    # Random name generation from first and last name
    name = f"{random.choice(first_names)} {random.choice(last_names)}"
    field = random.choice(research_fields)

    topic = prompt_scadsai_llm(f"""Generate an English PhD thesis topic for someone working at the institute '{field}'. Reply with only the title of the research project and nothing else.""", temperature=0.9)
    topic = topic.strip().replace("</s>", "").replace("\"", "")
    if ":" in topic and " " not in topic.split(":")[0]:
        topic = topic.split(":")[1]
    if "</thinking>" in topic:
        topic = topic.split("</thinking>")[1]

    print(len(topic), topic)
    
    profiles.append({
        "name": name,
        "research_field": field,
        "topic": topic
    })

136 Digital Archives, Embodied Knowledge, and the Preservation of Performance: A Study of Dance Documentation at the German Federal Archives
132 Microbial Secondary Metabolites and Narrative: Exploring Literary Representations of Fungal Discovery and Pharmaceutical Innovation.
109 Spatial Imaginaries of Ecological Transition: Narratives, Planning, and the Production of Futurity in Germany
84 Algorithmic Storytelling and the Evolution of Narrative Agency in Interactive Media.
121 The Literary Ecology of Scientific Illustration: Bridging Art, Science, and Specimen Culture in the Humboldtian Tradition
112 Narratives of Uncertainty: Climate Modeling, Public Discourse, and the Construction of Risk in the Anthropocene.
91 Resilience and Narrative: Constructing Collective Memory in Post-Disaster German Literature
139 The Palimpsest of Extinction: Tracing Ecological Grief and Anthropocene Narratives in Scientific and Literary Responses to Megafaunal Loss.
117 Computational Ecology of Digital A

Finally, we store the names, institutes and PhD topics to a csv file.

In [5]:
# Create DataFrame and save CSV
df = pd.DataFrame(profiles)
df.to_csv("phd_topics.csv", index=False)

df.head()

,name,research_field,topic
0,Taylor Reed,FIZ-KA - Leibniz-Institut für Informationsinfr...,"Digital Archives, Embodied Knowledge, and the ..."
1,Riley Jain,HKI - Leibniz-Institut für Naturstoff-Forschun...,Microbial Secondary Metabolites and Narrative:...
2,Taylor Adams,IÖR - Leibniz-Institut für ökologische Raument...,Spatial Imaginaries of Ecological Transition: ...
3,Devon Thomas,"IWM - Leibniz-Institut für Wissensmedien, Tübi...",Algorithmic Storytelling and the Evolution of ...
4,Alex Lee,MfN - Museum für Naturkunde - Leibniz-Institut...,The Literary Ecology of Scientific Illustratio...
